# DATA.ML.370 Exercise 3
Tay Jovan (153361337)

## Task 3.1
### Task 3.1a

In [2]:
frequent_items = []
for i in range(1, 101):
    support = 100 // i
    if support >= 5:
        frequent_items.append(i)

print(frequent_items)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


### Task 3.1b

In [3]:
def count_divisors(n):
    count = 0
    # divisors = []
    for b in range(1, n + 1):
        if n % b == 0:
            count += 1
            # divisors.append(b)
    # print(f'item i: {n}, divisors: {divisors}')
    return count

frequent_items = []
for i in range(1, 101):
    support = count_divisors(i)
    if support >= 5:
        frequent_items.append(i)
print(frequent_items)

[12, 16, 18, 20, 24, 28, 30, 32, 36, 40, 42, 44, 45, 48, 50, 52, 54, 56, 60, 63, 64, 66, 68, 70, 72, 75, 76, 78, 80, 81, 84, 88, 90, 92, 96, 98, 99, 100]


# Task 3.2
## Task 3.2.2

In [4]:
import os
import pyspark
from pyspark.sql import *
sc = pyspark.SparkContext("local", "FrequentItemsets")
os.system('wget http://files.grouplens.org/datasets/movielens/ml-1m.zip; unzip ml-1m.zip')

25/04/07 17:02:59 WARN Utils: Your hostname, jovan-laptop resolves to a loopback address: 127.0.1.1; using 192.168.8.175 instead (on interface wlp3s0)
25/04/07 17:02:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/07 17:03:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
--2025-04-07 17:03:05--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

     0K .......... .......... .......... .......... ..........  0%  122K 47s
    50K .......... .......... .......... .......... .....

Archive:  ml-1m.zip


2

In [5]:
import pandas as pd

ratings = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    header=None,
    engine="python",
    names=["UserID", "MovieID", "Rating", "Timestamp"]
)

movies = pd.read_csv(
    "ml-1m/movies.dat",
    sep="::",
    header=None,
    engine="python",
    names=["MovieID", "Title", "Genres"],
    encoding="ISO-8859-1"
)
users = pd.read_csv(
    "ml-1m/users.dat",
    sep="::",
    header=None,
    engine="python",
    names=["UserID", "Gender", "Age", "Occupation", "Zip-code"]
)


In [6]:

print(ratings.head())
print(movies.head())
print(users.head())

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455


## Task 3.2.3

In [8]:
from pyspark.mllib.fpm import FPGrowth

# Load movie names
lines = sc.textFile("ml-1m/movies.dat")
movienames = lines.map(lambda l: l.split("::")) \
                  .map(lambda p: (int(p[0]), p[1])) \
                  .collectAsMap()  # more efficient for lookups

# Load ratings and group by user
ratings = sc.textFile("ml-1m/ratings.dat")
usermovies = ratings.map(lambda l: l.split("::")) \
                    .map(lambda p: (int(p[0]), int(p[1]))) \
                    .groupByKey() \
                    .map(lambda p: list(p[1]))\
                    .cache()

usercount = usermovies.count()

# Start support from 0.2 and increment in steps (e.g. 0.05)
support = 0.2
step = 0.05
max_support_with_results = 0.0
frequent_sets = []

while True:
    model = FPGrowth.train(usermovies, minSupport=support, numPartitions=10)
    result = model.freqItemsets().collect()
    
    # Filter out itemsets with only one movie
    result = [x for x in result if len(x.items) > 1]

    if not result:
        print(f"\nNo results found at support={support}")
        break

    print(f"\nSupport: {support}")
    for x in result:
        movie_titles = [movienames[movie_id] for movie_id in x.items]
        print(movie_titles)
        print(f"Found in {x.freq} users out of {usercount}\n")
    
    max_support_with_results = support
    frequent_sets = result
    support += step

# After loop, show summary of highest support that gave results
print(f"\nHighest support that gave frequent sets: {max_support_with_results}")
print("Frequent sets at that support:")
for x in frequent_sets:
    movie_titles = [movienames[movie_id] for movie_id in x.items]
    print(movie_titles)
    print(f"Found in {x.freq} users out of {usercount}")


25/04/07 17:13:10 WARN FPGrowth: Input data is not cached.                      
25/04/07 17:13:48 WARN FPGrowth: Input data is not cached.                      



Support: 0.2
['E.T. the Extra-Terrestrial (1982)', 'Back to the Future (1985)']
Found in 1589 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Back to the Future (1985)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 1402 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Back to the Future (1985)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 1274 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Back to the Future (1985)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 1364 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Back to the Future (1985)', 'Star Wars: Episode VI - Return of the Jedi (1983)']
Found in 1305 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Back to the Future (1985)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 1241 users out of 6040

['E.T. the Extra-Te

25/04/07 17:14:07 WARN FPGrowth: Input data is not cached.                      



Support: 0.25
['E.T. the Extra-Terrestrial (1982)', 'Back to the Future (1985)']
Found in 1589 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 1753 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 1532 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 1714 users out of 6040

['E.T. the Extra-Terrestrial (1982)', 'Star Wars: Episode VI - Return of the Jedi (1983)']
Found in 1591 users out of 6040

['Men in Black (1997)', 'Back to the Future (1985)']
Found in 1565 users out of 6040

['Men in Black (1997)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 1770 users out of 6040

['Men in Black (1997)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 1565 user

25/04/07 17:14:14 WARN FPGrowth: Input data is not cached.                      



Support: 0.3
['Men in Black (1997)', 'Jurassic Park (1993)']
Found in 1939 users out of 6040

['Men in Black (1997)', 'Terminator 2: Judgment Day (1991)']
Found in 1855 users out of 6040

['Raiders of the Lost Ark (1981)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 1999 users out of 6040

['Raiders of the Lost Ark (1981)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 1972 users out of 6040

['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 2355 users out of 6040

['Star Wars: Episode V - The Empire Strikes Back (1980)', 'American Beauty (1999)']
Found in 1829 users out of 6040

['Being John Malkovich (1999)', 'American Beauty (1999)']
Found in 1884 users out of 6040

['Fargo (1996)', 'American Beauty (1999)']
Found in 1856 users out of 6040

['Star Wars: Episode VI - Return of the Jedi (1983)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 2228 users out of 6040

['Star Wa

25/04/07 17:14:18 WARN FPGrowth: Input data is not cached.                      



Support: 0.35
['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 2355 users out of 6040

['Star Wars: Episode VI - Return of the Jedi (1983)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 2228 users out of 6040


No results found at support=0.39999999999999997

Highest support that gave frequent sets: 0.35
Frequent sets at that support:
['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)']
Found in 2355 users out of 6040
['Star Wars: Episode VI - Return of the Jedi (1983)', 'Star Wars: Episode V - The Empire Strikes Back (1980)']
Found in 2228 users out of 6040
